In [3]:
import numpy as np
import soundfile as sf
from scipy.signal import correlate

# -----------------------------
# STEP 1: Load test signal
# -----------------------------
x, fs = sf.read('test_clap.wav')
x = x[:,0] if x.ndim > 1 else x  # use first channel if stereo

# -----------------------------
# STEP 2: Load 5 room recordings
# -----------------------------
recordings = []
for k in range(1, 6):
    y, fs = sf.read(f'test_clap{k}.wav')
    y = y[:,0] if y.ndim > 1 else y
    recordings.append(y)

# -----------------------------
# STEP 3: Time-align recordings
# -----------------------------
aligned_recordings = []
for y in recordings:

    corr = correlate(y, x, mode='full')
    shift = np.argmax(corr) - len(x) + 1
    y_aligned = np.roll(y, -shift)  # shift signal to align
    aligned_recordings.append(y_aligned)

min_len = min(len(y) for y in aligned_recordings)
Y = np.array([y[:min_len] for y in aligned_recordings])

# -----------------------------
# STEP 4: Average recordings (noise reduction)
# -----------------------------
y_avg = np.mean(Y, axis=0)

# -----------------------------
# STEP 5: Estimate impulse response
# -----------------------------
N = len(y_avg) + len(x)
X = np.fft.fft(x, N)
Yf = np.fft.fft(y_avg, N)

epsilon = 1e-6  # avoid division by zero
H = Yf / (X + epsilon)

# Impulse response
h = np.real(np.fft.ifft(H))

# -----------------------------
# STEP 6: Optional validation
# -----------------------------
y_est = np.convolve(x, h, mode='same')

# -----------------------------
# STEP 7: Save impulse response
# -----------------------------
sf.write('impulse_response4.wav', h, fs)

print("Impulse response saved as 'impulse_response.wav'.")

LibsndfileError: Error opening 'test_clap3.wav': System error.